In [9]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import MeCab
from gensim import corpora,matutils
reserve_tb = pd.read_csv("../awesomebook/data/reserve.csv")
production_tb = pd.read_csv("../awesomebook/data/production.csv")
customer_tb = pd.read_csv("../awesomebook/data/customer.csv")

C:\Users\UEDA\AppData\Local\Programs\Python\Python39\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [5]:
mecab = MeCab.Tagger ()

In [7]:
sentence = "太郎はこの本を女性に渡した。"
print(mecab.parse(sentence))

太郎	名詞,固有名詞,人名,名,*,*,太郎,タロウ,タロー
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
この	連体詞,*,*,*,*,*,この,コノ,コノ
本	名詞,一般,*,*,*,*,本,ホン,ホン
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
女性	名詞,一般,*,*,*,*,女性,ジョセイ,ジョセイ
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
渡し	動詞,自立,*,*,五段・サ行,連用形,渡す,ワタシ,ワタシ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
。	記号,句点,*,*,*,*,。,。,。
EOS



In [14]:
reserve_tb['reserve_datetime'] = pd.to_datetime(reserve_tb['reserve_datetime'],format = '%Y-%m-%d %H:%M:%S')

In [15]:
reserve_tb['checkin_date'] = pd.to_datetime(reserve_tb['checkin_date'],format = '%Y-%m-%d %H:%M:%S')

In [17]:
reserve_tb['reserve_datetime'].dt.year*12+reserve_tb['reserve_datetime'].dt.month-reserve_tb['checkin_date'].dt.year*12-reserve_tb['checkin_date'].dt.month

0       0
1       0
2      -1
3       0
4       0
       ..
4025   -1
4026   -1
4027   -1
4028   -1
4029   -1
Length: 4030, dtype: int64

In [26]:
reserve_tb

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
0,r1,h_75,c_1,2016-03-07 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200
1,r2,h_219,c_1,2016-07-17 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600
2,r3,h_179,c_1,2016-09-25 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600
3,r4,h_214,c_1,2017-03-09 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400
4,r5,h_16,c_1,2017-09-06 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100
...,...,...,...,...,...,...,...,...,...
4025,r4026,h_129,c_999,2017-06-28 23:00:02,2017-07-10,09:30:00,2017-07-11,2,16000
4026,r4027,h_97,c_999,2017-09-30 05:24:57,2017-10-09,10:30:00,2017-10-10,2,41800
4027,r4028,h_27,c_999,2018-03-15 05:01:45,2018-04-02,11:30:00,2018-04-04,2,74800
4028,r4029,h_48,c_1000,2016-04-17 15:20:17,2016-05-10,09:30:00,2016-05-13,4,540000


In [27]:
import datetime
reserve_tb.dtypes

reserve_id                  object
hotel_id                    object
customer_id                 object
reserve_datetime    datetime64[ns]
checkin_date        datetime64[ns]
checkin_time                object
checkout_date               object
people_num                   int64
total_price                  int64
dtype: object

In [28]:
reserve_tb['reserve_datetime_1d'] = reserve_tb['reserve_datetime'] + datetime.timedelta(days =1)

In [29]:
reserve_tb

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,reserve_datetime_1d
0,r1,h_75,c_1,2016-03-07 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,2016-03-08 13:09:42
1,r2,h_219,c_1,2016-07-17 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,2016-07-18 23:39:55
2,r3,h_179,c_1,2016-09-25 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,2016-09-26 10:03:17
3,r4,h_214,c_1,2017-03-09 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,2017-03-10 03:20:10
4,r5,h_16,c_1,2017-09-06 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,2017-09-07 19:50:37
...,...,...,...,...,...,...,...,...,...,...
4025,r4026,h_129,c_999,2017-06-28 23:00:02,2017-07-10,09:30:00,2017-07-11,2,16000,2017-06-29 23:00:02
4026,r4027,h_97,c_999,2017-09-30 05:24:57,2017-10-09,10:30:00,2017-10-10,2,41800,2017-10-01 05:24:57
4027,r4028,h_27,c_999,2018-03-15 05:01:45,2018-04-02,11:30:00,2018-04-04,2,74800,2018-03-16 05:01:45
4028,r4029,h_48,c_1000,2016-04-17 15:20:17,2016-05-10,09:30:00,2016-05-13,4,540000,2016-04-18 15:20:17


In [12]:
reserve_tb = reserve_tb[(abs(reserve_tb['total_price']-np.mean(reserve_tb['total_price']))/np.std(reserve_tb['total_price']))<=3].reset_index()

In [31]:
reserve_tb.dtypes

reserve_id                     object
hotel_id                       object
customer_id                    object
reserve_datetime       datetime64[ns]
checkin_date           datetime64[ns]
checkin_time                   object
checkout_date                  object
people_num                      int64
total_price                     int64
reserve_datetime_1d    datetime64[ns]
dtype: object

In [32]:
def to_season(month):
    season = 'winter'
    if 3<=month<=5:
        season='spring'
    elif 6<=month<=8:
        season='summer'
    elif 9<=month<=11:
        season='fall'
    return season


In [36]:
reserve_tb['season'] = reserve_tb['reserve_datetime'].dt.month.apply(to_season)

In [37]:
reserve_tb

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,reserve_datetime_1d,season
0,r1,h_75,c_1,2016-03-07 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,2016-03-08 13:09:42,spring
1,r2,h_219,c_1,2016-07-17 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,2016-07-18 23:39:55,summer
2,r3,h_179,c_1,2016-09-25 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,2016-09-26 10:03:17,fall
3,r4,h_214,c_1,2017-03-09 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,2017-03-10 03:20:10,spring
4,r5,h_16,c_1,2017-09-06 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,2017-09-07 19:50:37,fall
...,...,...,...,...,...,...,...,...,...,...,...
4025,r4026,h_129,c_999,2017-06-28 23:00:02,2017-07-10,09:30:00,2017-07-11,2,16000,2017-06-29 23:00:02,summer
4026,r4027,h_97,c_999,2017-09-30 05:24:57,2017-10-09,10:30:00,2017-10-10,2,41800,2017-10-01 05:24:57,fall
4027,r4028,h_27,c_999,2018-03-15 05:01:45,2018-04-02,11:30:00,2018-04-04,2,74800,2018-03-16 05:01:45,spring
4028,r4029,h_48,c_1000,2016-04-17 15:20:17,2016-05-10,09:30:00,2016-05-13,4,540000,2016-04-18 15:20:17,spring
